# Case for MacPaw Internship (MarTech & Analytics)

## Task #1: Digital marketing calculations

In [ ]:
week_subscript_cost = int(input("Weekly subscription cost "))
app_install_expected = float(input("App installers expected to choose the subscription option "))
life_expect_paid_user = int(input("Average life expectancy of a paid user "))
coeff_organic_install = float(input("The coefficient of free (organic) installations "))
cpi = float(input("Average cost of one installation "))
refund_rate = float(input("Refund rate (unsubscription, refund) "))
app_store_commiss = float(input("App Store commission "))
net_profit = int(input("Expected net profit "))

installs = net_profit / (life_expect_paid_user * 
                         week_subscript_cost * 
                         app_install_expected * 
                         ((1 - app_store_commiss) * (1 - refund_rate)) - 
                         (1 - coeff_organic_install) * 
                         cpi)
total_cpi = (installs * (1 - coeff_organic_install)) * cpi

print()
print("It takes {:.0f} installs to get $ {:.0f} in net profit. We need to spend $ {:.2f} to achieve this goal.".format(installs, net_profit, total_cpi))

## Task #2: Exchange rate notifier

Unfortunately, I haven't completed this task. But it was so challenging and interesting for me that I want to share the lessons I learned and conclusions I made while getting though it.

<b><center>Exposition</center></b><br />
I'm pretty much new to programming and especially IT domain. All my experience here was limited to creating basic web sites with HTML and CSS. I learned Python basics a couple of weeks ago, thanks to online courses from Bioinformatics Institute and Yandex. Thus, I didn't even know how to use APIs and that it's possible to create programs for Telegram bots with Python.

<b><center>Introduction</center></b><br />
So, at first, I was slightly confused by this task and thought it's impossible even to start it. Fortunately, our collective intellect (Internet, I mean) had a bunch of tutorials, examples and info that helped me to write the script you could find in the next section.<br />
Bot name is Notifiex (link - @notifiex_bot).
I used <b>pyTelegramBotAPI</b> to connect to my bot, perform commands like send, reply to messages, set starting commands as well as to get information about users: first name, last name and user id; <b>BeautifulSoup</b> library to parse the exchange rate from Google Search; <b>MySQL</b> database service to store the information about the users and the last exchange rate showed them.

<b><center>Сulmination</center></b>

In [ ]:
import sys
import telebot
import requests
from bs4 import BeautifulSoup
import time
import mysql.connector
from mysql.connector import errorcode

# Connect to telegram-bot through TOKEN taken from BotFather
bot = telebot.TeleBot('1656087115:AAHUMv9M4uKcnrdH2up6nHPGTNKKePTAYtg')

# Connect to database and stop the program when something went wrong
try:
    db = mysql.connector.connect(
        host="localhost",
        user="root",
        password="**************",
        port="3306",
        database="users"
)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
        sys.exit()
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
        sys.exit()
    else:
        print(err)
        sys.exit()

cursor = db.cursor()  # used to execute statements to communicate with the MySQL database.
# Using the methods of it (e.g. fetchone()) you can execute SQL statements, fetch data from the result sets, call procedures.


class User:
    def __init__(self, first_name):
        self.first_name = first_name
        self.last_name = ''
        self.ex_rate = ''


# Google's page parser
class Currency:
    USD_UAH = 'https://www.google.com/search?q=%D0%BA%D1%83%D1%80%D1%81+%D0%B4%D0%BE%D0%BB%D0%BB%D0%B0%D1%80%D0%B0&oq=%D0%BA%D1%83%D1%80%D1%81&aqs=chrome.1.69i57j0i433j69i61j69i65l3j69i61l2.4696j0j1&sourceid=chrome&ie=UTF-8'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'}

    current_converted_price = 0

    def __init__(self):
        self.current_converted_price = float(self.get_currency_price())

    def get_currency_price(self):
        full_page = requests.get(self.USD_UAH, headers=self.headers)
        soup = BeautifulSoup(full_page.content, 'html.parser')
        convert = soup.findAll("span", {"class": "DFlfde SwHCTb", "data-precision": "2"})
        return convert[0].text.replace(",", ".")


currency = Currency()


@bot.message_handler(commands=['start', 'help'])
def start(message):
    try:
        send_mess = f"Hi!\n" \
                    f"I'm Notifiex, your personal exchange rate notifier!\n" \
                    f"I'll send you exchange rate for USD/UAH pair whenever it has grown since the last check.\n" \
                    f"Now, 1 USD = <b>{currency.get_currency_price()}</b> UAH"
        bot.send_message(message.chat.id, send_mess, parse_mode='html')

        user_id = message.from_user.id
        user = User(message.from_user.first_name)
        user.last_name = message.from_user.last_name
        user.ex_rate = float(currency.get_currency_price())

        # Register new user to the database
        sql = "INSERT INTO users (first_name, last_name, user_id, ex_rate) VALUES (%s, %s, %s, %s)"
        val = (user.first_name, user.last_name, user_id, user.ex_rate)
        cursor.execute(sql, val)
        db.commit()  # Since by default Connector/Python does not autocommit, it is important to call this method after
        # every transaction that modifies data for tables that use transactional storage engines.

        time.sleep(24*3600)
        # Looping the exchange rate checking
        while True:
            actual_price = float(currency.get_currency_price())
            user_id = message.from_user.id

            # Check the latest saved exchange rate
            sql = "SELECT ex_rate FROM users WHERE user_id = %s"
            val = (user_id, )
            cursor.execute(sql, val)
            result = cursor.fetchone()

            if actual_price > result[0]:
                rate_up = f"Hey, rate has grown up!\nNow, 1 USD costs <b>{actual_price}</b> UAH"
                bot.send_message(message.from_user.id, rate_up, parse_mode='html')

                # Rewriting the exchange rate value when it's grown up
                sql = "UPDATE users SET ex_rate = %s WHERE user_id = %s"
                val = (actual_price, user_id)
                cursor.execute(sql, val)
                db.commit()

            # Stop the program for a while
            time.sleep(24*3600)  # once a day
    except Exception as e:
        bot.send_message(message.chat.id, f"You've already started me or something went wrong :(\n"
                                          f"Then, please, try later.", parse_mode='html')


@bot.message_handler(content_types=['text'])
def mess(message):
    try:
        if 'rate' in message.text:
            actual_price = currency.get_currency_price()
            send_mess = f"Actual rate: 1 USD = <b>{actual_price}</b> UAH"
            bot.send_message(message.chat.id, send_mess, parse_mode='html')
            time.sleep(24 * 3600)
            while True:
                actual_price = float(currency.get_currency_price())
                user_id = message.from_user.id

                sql = "SELECT ex_rate FROM users WHERE user_id = %s"
                val = (user_id,)
                cursor.execute(sql, val)
                result = cursor.fetchone()

                if actual_price > result[0]:
                    rate_up = f"Hey, rate has grown up!\nNow, 1 USD costs <b>{actual_price}</b> UAH"
                    bot.send_message(message.from_user.id, rate_up, parse_mode='html')

                    sql = "UPDATE users SET ex_rate = %s WHERE user_id = %s"
                    val = (actual_price, user_id)
                    cursor.execute(sql, val)
                    db.commit()

                time.sleep(24 * 3600)  # once a day
        else:
            sorry_mess = f"Sorry, I can't understand you"
            bot.send_message(message.chat.id, sorry_mess, parse_mode='html')
            time.sleep(24 * 3600)
            while True:
                actual_price = float(currency.get_currency_price())
                user_id = message.from_user.id

                sql = "SELECT ex_rate FROM users WHERE user_id = %s"
                val = (user_id,)
                cursor.execute(sql, val)
                result = cursor.fetchone()

                if actual_price > result[0]:
                    rate_up = f"Hey, rate has grown up!\nNow, 1 USD costs <b>{actual_price}</b> UAH"
                    bot.send_message(message.from_user.id, rate_up, parse_mode='html')

                    sql = "UPDATE users SET ex_rate = %s WHERE user_id = %s"
                    val = (actual_price, user_id)
                    cursor.execute(sql, val)
                    db.commit()

                time.sleep(24 * 3600)  # once a day
    except Exception as e:
        bot.send_message(message.chat.id, 'Sorry, something went wrong :( \n Please, try later.')


if __name__ == '__main__':  # if current file - main
    bot.polling(none_stop=True, interval=0)  # then start bot

<b><center>Denouement</center></b><br />
I understand that program works on a very basic level and with many limitations:
1. There are only two ways user can interact with the bot;
2. When user calls "/start" command twice or more, program stops;
3. I believe, there is much more better way to send message automatically than using "while" and "time.sleep" functions;
4. As for me, parser or database recorder doesn't work properly, that's why there are problems with comparison of values and updating them in database;
5. Program doesn't work in standalone mode, because I didn't deploy it on server (Heroku, by instance)
5. ... (there're some I couldn's see/understend).

As I said, task was challenging for me, particularly when I didn't know why on some of the steps the program didn't work. For example, I thought that it's possible to save the latest exchange rate showed to user without database, just with dictionary. And I spent quite much time looking for or thinking about the solution. After all, I reread the task and realized that when there was "data storage" keyword in it, then I was inventing a bicycle. When I used database to solve the issue, the program, by halves, started to work the way it should.<br />
Such challenges weren't uncommon, but it gave me a lot of lessons:
1. The problem/situation/task doesn't look so challenging if it's divided on small objects;
2. The better you search, the faster you find/come up with the solution. For the better search, high-quality questions needed. Such questions are possible when you look at the problem from different angles.
3. It's always make sense to visualise the problem.
4. Every API has a documentation with pretty clear explanatory text, but it often shows common examples (I don't know, maybe in other documentations it's not so) and for a deeper understanding you need to search.
5. Never give up on the task.
6. If you gave up, remember that actually you didn't.
7. If you gave up for the third time - take a break.
8. Rest helps with solving the problem.
9. Programming is a very hard work, when you don't have technical background and mentor. Actually, it's one of the greatest resons I've not completed the task. But it's possible to learn programming with the proper motivation and desire.
10. ... (and also a lot of different stuff related to code, databases, syntax etc.)

<b><center>Epilogue</center></b>
The story is over. I appreciate that you read to the end!<br />
Thanks to all the people, who organized MacPaw Internship, especially those, who set this task. It was interesting time for me, because I: tested my limits, find out new things, become better.<br />
<center>Wish you all good!</center>